<a href="https://colab.research.google.com/github/GUSTAVObbarbosa/ferramentas_survey123/blob/main/Atualizando_pasta_media.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Acessando os dados

Neste exemplo os dados estão em Excel em uma diretório no Google Drive.

O ideal é sempre termos uma pasta oficial com a planilha de recomendação técnica. Aqui, arquivo é um excel, que pode ser gerado de forma manual. Mas caso, esse processo já esteja automatizado, podemos realizá-lo direto no Python, ou caso seja gerado por algum sistema, podemos conectar o output desse sistema (recomendação final) no início do processo.

O foco aqui é apresentar um processo simples de tratamento de dados e disponibilizaçao no Survey123.

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

#Planilha de recomendação técnica em .xlsx
recomendacao_tecnica = '/content/drive/MyDrive/Recomendação técnica/Recomendação Técnica.xlsx' 

#Ler os arquivos no Python
df_recomendacao = pd.read_excel(recomendacao_tecnica,
                                converters={'Talhão':str})
df_recomendacao.head()

,Fazenda,Talhão,Atividade,Produto,Dose,Unidade,Data prevista,Status atividade
0,AROEIRA,001,Calagem,A,10.0,t/ha,2022-07-01,Pendente
1,AROEIRA,001,Adubação de base,B,300.0,kg/ha,2022-07-20,Pendente
2,AROEIRA,002,Adubação de base,B,250.0,kg/ha,2022-08-31,Pendente
3,AROEIRA,003,Adubação de cobertura,C,430.0,kg/ha,2022-07-05,Pendente
4,RIO PIRACICABA,002,Adubação de base,B,280.0,kg/ha,2022-08-02,Pendente


##Tratamento dos dados

Aqui vamos tratar os dados de forma a padronizar e disponibilizar os dados prontos.

In [ ]:
#Padronizar nomes das atividades (letras maiúsculas) e retirar espaços
conditions = [df_recomendacao['Atividade']=='Calagem',
              df_recomendacao['Atividade']=='Adubação de base',
              df_recomendacao['Atividade']=='Adubação de cobertura']

values = ['CALAGEM',
          'ADUBACAO_BASE',
          'ADUBACAO_COBERTURA']

df_recomendacao['atividade_padrao'] = np.select(conditions, values)

#Retirar espaços em branco na coluna Fazenda
df_recomendacao['Fazenda'] = df_recomendacao['Fazenda'].str.replace(' ','_')

#Criar coluna com ID
df_recomendacao['ID'] = df_recomendacao['Fazenda'] + "-" + df_recomendacao['Talhão'] + "-" + df_recomendacao['atividade_padrao'] 

#Padronizar data no formato "dd/mm/aaaa"
df_recomendacao['Data prevista'] = pd.to_datetime(df_recomendacao['Data prevista']).dt.strftime("%d/%m/%Y")

#Dosagem em número inteiro
df_recomendacao['Dose'] = df_recomendacao['Dose'].astype(int)

In [ ]:
#Planilha após tratamento
df_recomendacao.head()

,Fazenda,Talhão,Atividade,Produto,Dose,Unidade,Data prevista,Status atividade,atividade_padrao,ID
0,AROEIRA,001,Calagem,A,10,t/ha,01/07/2022,Pendente,CALAGEM,AROEIRA-001-CALAGEM
1,AROEIRA,001,Adubação de base,B,300,kg/ha,20/07/2022,Pendente,ADUBACAO_BASE,AROEIRA-001-ADUBACAO_BASE
2,AROEIRA,002,Adubação de base,B,250,kg/ha,31/08/2022,Pendente,ADUBACAO_BASE,AROEIRA-002-ADUBACAO_BASE
3,AROEIRA,003,Adubação de cobertura,C,430,kg/ha,05/07/2022,Pendente,ADUBACAO_COBERTURA,AROEIRA-003-ADUBACAO_COBERTURA
4,RIO_PIRACICABA,002,Adubação de base,B,280,kg/ha,02/08/2022,Pendente,ADUBACAO_BASE,RIO_PIRACICABA-002-ADUBACAO_BASE


#Salvando arquivo final no Survey123

In [ ]:
#Salvar planilha de recomendação em .csv na pasta do Survey123
diretorio_survey123 = 'C:/Users/nome_usuario/ArcGIS/My Survey Designs/Recomendação Técnica/media/recomendacao.csv'

df_recomendacao.to_csv(diretorio_survey123)